In [13]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [20]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('/opt/ml/detection/baseline/mmdetection/work_dirs/swinB_fold_3/swin_b_cascade.py')
cfg = Config.fromfile('/opt/ml/detection/baseline/mmdetection/work_dirs/train_pseudo_fold_4/swin_b_cascade.py')

root='/opt/ml/detection/dataset/'

epoch = ''

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.seed=2021
cfg.gpu_ids = [0]

for i in range(1, 20):
    # dir_path = f'./work_dirs/cascade_rcnn_trash_{i}'
    dir_path = f'./work_dirs/cascade_rcnn_trash_pseudo_{i}'
    if os.path.isdir(dir_path):
        cfg.work_dir = dir_path
        break
cfg.work_dir = './work_dirs/train_pseudo_fold_4'
print(cfg.work_dir)

for i in range(3):
    cfg.model.roi_head.bbox_head[i].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

./work_dirs/swinB_fold_3


In [21]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=2, #cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [22]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/swinB_fold_3/best_bbox_mAP_epoch_12.pth


In [23]:
output = single_gpu_test(model, data_loader, show_score_thr=0.001) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 1.5 task/s, elapsed: 3313s, ETA:     0s

In [24]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        # print(out[j])
        # if len(out[j]) == 5:
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


,PredictionString,image_id
0,0 0.17971432 640.6211 253.99802 730.36255 346....,test/0000.jpg
1,0 0.6153053 346.42456 249.45615 754.01025 695....,test/0001.jpg
2,0 0.6597554 662.86066 290.18372 742.1502 324.9...,test/0002.jpg
3,0 0.05260449 0.0 6.5856247 676.1471 302.91248 ...,test/0003.jpg
4,0 0.7461374 204.05621 345.67365 872.739 766.95...,test/0004.jpg
